<a href="https://colab.research.google.com/github/Erenkyz/Movie-Revenue-Prediction-with-Transformer-on-Tabular-Data/blob/main/Movie_revenue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

## DATA PROCESSING & CLEANING

In [2]:
movies_df = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

total_samples = len(movies_df)
print("Total samples:", total_samples)

total_samples = len(credits)
print("Total samples:", total_samples)

Total samples: 4803
Total samples: 4803


In [3]:
movies_df = movies_df.merge(credits[['movie_id', 'cast', 'crew']], left_on='id', right_on='movie_id', how='left')

In [4]:
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [6]:
total_samples = len(movies_df)
print("Total samples:", total_samples)

Total samples: 4803


In [7]:
movies_df = movies_df[["budget","genres","production_companies","keywords","original_language","popularity","release_date","revenue","runtime","vote_average","movie_id","cast","crew"]]


In [8]:
print(movies_df.isnull().sum())

budget                  0
genres                  0
production_companies    0
keywords                0
original_language       0
popularity              0
release_date            1
revenue                 0
runtime                 2
vote_average            0
movie_id                0
cast                    0
crew                    0
dtype: int64


In [9]:
print(movies_df.isnull().sum())


movies_df['budget'] = movies_df['budget'].fillna(movies_df['budget'].median())
movies_df['revenue'] = movies_df['revenue'].fillna(movies_df['revenue'].median())
movies_df['runtime'] = movies_df['runtime'].fillna(movies_df['runtime'].median())
movies_df['popularity'] = movies_df['popularity'].fillna(movies_df['popularity'].median())
movies_df['vote_average'] = movies_df['vote_average'].fillna(movies_df['vote_average'].median())

movies_df['original_language'] = movies_df['original_language'].fillna('en')
movies_df['genres'] = movies_df['genres'].fillna('[]')
movies_df['production_companies'] = movies_df['production_companies'].fillna('[]')
movies_df['keywords'] = movies_df['keywords'].fillna('[]')
movies_df['cast'] = movies_df['cast'].fillna('[]')
movies_df['crew'] = movies_df['crew'].fillna('[]')


movies_df['release_date'] = movies_df['release_date'].fillna('2000-01-01')
movies_df.dropna(inplace=True)

budget                  0
genres                  0
production_companies    0
keywords                0
original_language       0
popularity              0
release_date            1
revenue                 0
runtime                 2
vote_average            0
movie_id                0
cast                    0
crew                    0
dtype: int64


In [10]:
print(movies_df["genres"])

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object


In [11]:
import ast

def parse_and_join_genres(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        if isinstance(genres, list):
          names = sorted([g["name"] for g in genres])
          return "|".join(names) if names else "None"
    except:
        return "None"

movies_df["genre_group"] = movies_df["genres"].apply(parse_and_join_genres)

In [12]:
def extract_main_company(companies_str):
    try:
        companies = ast.literal_eval(companies_str)
        if companies:
          return companies[0]["name"]
    except:
      return "Unknown"

movies_df["main_company"] = movies_df['production_companies'].apply(extract_main_company)

In [13]:
def join_keywords(kw_str):
    try:
      keywords = ast.literal_eval(kw_str)
      return "|".join(sorted([k["name"] for k in keywords]))
    except:
      return "None"

movies_df["keyword_combo"] = movies_df["keywords"].apply(join_keywords)

In [14]:
def get_top_cast_names(cast_str, top_n=3):
    try:
        cast_list = ast.literal_eval(cast_str)
        names = [actor["name"] for actor in cast_list[:top_n]]
        return "|".join(names)
    except:
        return "Unknown"

movies_df["top_cast"] = movies_df["cast"].apply(lambda x: get_top_cast_names(x, 3))


In [15]:
def get_director(crew_str):
    try:
        crew_list = ast.literal_eval(crew_str)
        for member in crew_list:
            if member.get("job") == "Director":
                return member.get("name")
    except:
        return "Unknown"
    return "Unknown"

movies_df["director"] = movies_df["crew"].apply(get_director)


In [16]:
movies_df.drop(columns=['genres', 'production_companies', 'keywords', 'cast', 'crew'], inplace=True)

In [17]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')

movies_df["release_year"] = movies_df["release_date"].dt.year
movies_df["release_month"] = movies_df["release_date"].dt.month
movies_df["release_dayofweek"] = movies_df["release_date"].dt.dayofweek

movies_df = movies_df.drop(columns=["release_date"])

In [18]:
movies_df["budget"] = movies_df["budget"].replace(0, movies_df["budget"].median())
scaler = MinMaxScaler()
movies_df["scaled_budget"] = scaler.fit_transform(movies_df[["budget"]])

movies_df["log_budget"] = np.log1p(movies_df["budget"])


In [19]:
label_encoders = {}
for col in ["original_language", "release_year", "release_month", "release_dayofweek", "genre_group"]:
    le = LabelEncoder()
    movies_df[f"encoded_{col}"] = le.fit_transform(movies_df[col])
    label_encoders[col] = le

In [20]:
movies_df["scaled_popularity"] = scaler.fit_transform(movies_df[["popularity"]])

In [21]:
movies_df["scaled_runtime"] = scaler.fit_transform(movies_df[["runtime"]])


In [22]:
movies_df["scaled_vote_average"] = scaler.fit_transform(movies_df[["vote_average"]])


In [23]:
movies_df["revenue_class"] = pd.qcut(movies_df["revenue"], q=3, labels=["Low", "Medium", "High"])

label_map = {"Low": 0, "Medium": 1, "High": 2}
movies_df["encoded_revenue_class"] = movies_df["revenue_class"].map(label_map)



In [24]:
movies_df.columns

Index(['budget', 'original_language', 'popularity', 'revenue', 'runtime',
       'vote_average', 'movie_id', 'genre_group', 'main_company',
       'keyword_combo', 'top_cast', 'director', 'release_year',
       'release_month', 'release_dayofweek', 'scaled_budget', 'log_budget',
       'encoded_original_language', 'encoded_release_year',
       'encoded_release_month', 'encoded_release_dayofweek',
       'encoded_genre_group', 'scaled_popularity', 'scaled_runtime',
       'scaled_vote_average', 'revenue_class', 'encoded_revenue_class'],
      dtype='object')

In [25]:
movies_df = movies_df.drop(columns=["budget","original_language","popularity","revenue","runtime","vote_average",'release_month','release_dayofweek','release_year'])

In [26]:
movies_df["encoded_keywords"] = LabelEncoder().fit_transform(movies_df["keyword_combo"])


In [27]:
director_freq = movies_df["director"].value_counts().to_dict()
movies_df["director_freq"] = movies_df["director"].map(director_freq)


company_freq = movies_df["main_company"].value_counts().to_dict()
movies_df["main_company_freq"] = movies_df["main_company"].map(company_freq)
movies_df["main_company_freq"] = movies_df["main_company_freq"].fillna(0)


cast_freq = movies_df["top_cast"].value_counts().to_dict()
movies_df["top_cast_freq"] = movies_df["top_cast"].map(cast_freq)


In [28]:

movies_df["budget_genre_combo"] = movies_df["scaled_budget"] * movies_df["encoded_genre_group"]


movies_df["popularity_year_combo"] = movies_df["scaled_popularity"] * movies_df["encoded_release_year"]


In [29]:
movies_df = movies_df.drop(columns=["movie_id","genre_group","main_company","top_cast","director","keyword_combo"])

In [30]:
movies_df.to_csv("movies_df_with_encoded_columns.csv", index = False)

In [31]:
movies_df["encoded_revenue_class"].value_counts()

,count
encoded_revenue_class,
0,1601
1,1601
2,1601


In [32]:
movies_df.head()

,scaled_budget,log_budget,encoded_original_language,encoded_release_year,encoded_release_month,encoded_release_dayofweek,encoded_genre_group,scaled_popularity,scaled_runtime,scaled_vote_average,revenue_class,encoded_revenue_class,encoded_keywords,director_freq,main_company_freq,top_cast_freq,budget_genre_combo,popularity_year_combo
0,0.623684,19.283571,7,81,11,3,81,0.171815,0.479290,0.72,High,2,173,7,22.0,1,50.518421,13.916976
1,0.789474,19.519293,7,79,4,5,77,0.158846,0.500000,0.69,High,2,481,7,114.0,2,60.789474,12.548836
2,0.644737,19.316769,7,87,9,0,40,0.122635,0.437870,0.63,High,2,1924,7,200.0,1,25.789474,10.669233
3,0.657895,19.336971,7,84,6,0,161,0.128272,0.488166,0.76,High,2,2067,8,16.0,1,105.921052,10.774885
4,0.684211,19.376192,7,84,2,2,99,0.050169,0.390533,0.61,High,2,120,4,114.0,1,67.736842,4.214192


##1. Split Features and Target

In [33]:
X = movies_df.drop(columns=['revenue_class', 'encoded_revenue_class'])
y = movies_df['encoded_revenue_class']

In [34]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp)

In [35]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [36]:
class MovieDataset(Dataset):
    def __init__(self, features, labels):
        self.X = torch.tensor(features, dtype=torch.float32)
        self.y = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = MovieDataset(X_train_scaled, y_train)
val_dataset = MovieDataset(X_val_scaled, y_val)
test_dataset = MovieDataset(X_test_scaled, y_test)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)


In [37]:
class TabularTransformer(nn.Module):
    def __init__(self, input_dim, model_dim=64, num_classes=3, num_heads=4, num_layers=2, dropout=0.1):
        super(TabularTransformer, self).__init__()

        self.input_dim = input_dim
        self.model_dim = model_dim


        self.input_projection = nn.Linear(input_dim, model_dim)




        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=128,
            dropout=dropout,
            activation='relu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)


        self.classifier = nn.Sequential(
            nn.Linear(model_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):

        x = x.unsqueeze(1)


        x = self.input_projection(x)


        x = self.transformer_encoder(x)


        x = x.squeeze(1)


        out = self.classifier(x)
        return out


In [38]:
model = TabularTransformer(input_dim=X_train.shape[1])

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)


In [40]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0

    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == y_batch).sum().item()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [41]:
@torch.no_grad()
def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0

    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)

        loss = criterion(outputs, y_batch)
        running_loss += loss.item() * X_batch.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == y_batch).sum().item()

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = correct / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [42]:
n_epochs = 20
best_val_acc = 0.0

for epoch in range(n_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")

    print(f"Epoch {epoch+1}/{n_epochs} | Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


Epoch 1/20 | Train Loss: 0.8328, Acc: 0.6033 | Val Loss: 0.6851, Acc: 0.6982
Epoch 2/20 | Train Loss: 0.7121, Acc: 0.6775 | Val Loss: 0.6544, Acc: 0.6982
Epoch 3/20 | Train Loss: 0.6762, Acc: 0.6928 | Val Loss: 0.6288, Acc: 0.7149
Epoch 4/20 | Train Loss: 0.6861, Acc: 0.6855 | Val Loss: 0.6393, Acc: 0.7180
Epoch 5/20 | Train Loss: 0.6820, Acc: 0.6852 | Val Loss: 0.6252, Acc: 0.7253
Epoch 6/20 | Train Loss: 0.6542, Acc: 0.7084 | Val Loss: 0.6280, Acc: 0.7242
Epoch 7/20 | Train Loss: 0.6379, Acc: 0.7171 | Val Loss: 0.6241, Acc: 0.7336
Epoch 8/20 | Train Loss: 0.6594, Acc: 0.7095 | Val Loss: 0.6335, Acc: 0.7138
Epoch 9/20 | Train Loss: 0.6340, Acc: 0.7123 | Val Loss: 0.6336, Acc: 0.7138
Epoch 10/20 | Train Loss: 0.6283, Acc: 0.7331 | Val Loss: 0.6095, Acc: 0.7492
Epoch 11/20 | Train Loss: 0.6474, Acc: 0.6970 | Val Loss: 0.6198, Acc: 0.7347
Epoch 12/20 | Train Loss: 0.6223, Acc: 0.7320 | Val Loss: 0.6227, Acc: 0.7419
Epoch 13/20 | Train Loss: 0.6094, Acc: 0.7237 | Val Loss: 0.6181, Acc: 0.

In [43]:
model.load_state_dict(torch.load("best_model.pt"))
test_loss, test_acc = evaluate(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_acc:.4f}")

Test Loss: 0.6564, Accuracy: 0.6993


In [44]:
model.eval()
all_preds, all_targets = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(y_batch.numpy())

print(classification_report(all_targets, all_preds, target_names=["Low", "Medium", "High"]))


              precision    recall  f1-score   support

         Low       0.74      0.70      0.72       320
      Medium       0.56      0.61      0.59       320
        High       0.81      0.79      0.80       321

    accuracy                           0.70       961
   macro avg       0.71      0.70      0.70       961
weighted avg       0.71      0.70      0.70       961



In [45]:
revenue_class_map = {0: "Low", 1: "Medium", 2: "High"}

def predict_movie(model, X_input):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).to(device)
        output = model(X_tensor)
        pred_class = output.argmax(dim=1).item()
        return revenue_class_map[pred_class]

predictions = []
for i in range(5):
    sample_features = X_test_scaled[i]
    actual_class = y_test.iloc[i]
    predicted_class = predict_movie(model, sample_features)
    print(f" Movie #{i+1} → Predicted: {predicted_class} | Actual: {revenue_class_map[actual_class]}")


 Movie #1 → Predicted: Low | Actual: Low
 Movie #2 → Predicted: Low | Actual: Low
 Movie #3 → Predicted: Medium | Actual: Medium
 Movie #4 → Predicted: Low | Actual: Low
 Movie #5 → Predicted: Low | Actual: Medium


In [46]:
for i in range(5):
    sample_features = X_test_scaled[i]
    actual_class = y_test.iloc[i]
    predicted_class = predict_movie(model, sample_features)

    print(f"Movie {i+1}:")
    print(f"   Predicted: {predicted_class}")
    print(f"   Actual:    {revenue_class_map[actual_class]}")
    print()

Movie 1:
   Predicted: Low
   Actual:    Low

Movie 2:
   Predicted: Low
   Actual:    Low

Movie 3:
   Predicted: Medium
   Actual:    Medium

Movie 4:
   Predicted: Low
   Actual:    Low

Movie 5:
   Predicted: Low
   Actual:    Medium



In [47]:
revenue_class_map = {0: "Low", 1: "Medium", 2: "High"}

def predict_movie(model, X_input):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).to(device)
        output = model(X_tensor)
        pred_class = output.argmax(dim=1).item()
        return revenue_class_map[pred_class]


In [48]:
new_movie_raw = {
    "scaled_budget": 0.43,
    "encoded_language": 3,
    "scaled_popularity": 0.2,
    "encoded_release_year": 30,
    "encoded_release_month": 5,
    "encoded_release_dayofweek": 3,
    "scaled_runtime": 0.5,
    "scaled_vote_average": 0.8,

}


columns = X_train.columns.tolist()
new_movie_df = pd.DataFrame([new_movie_raw], columns=columns)



new_movie_scaled = scaler.transform(new_movie_df)



print("Predicted revenue class:", predict_movie(model, new_movie_scaled[0]))


Predicted revenue class: Low
